In [22]:
import math
import numpy as np
import pandas as pd
from typing import List

Let $S(i, j)$ and $S(i+1, j)$ denote the shfit factors of a given set of submatrices.
The Z messages associated with a submatrix are mapped to a bunch of row chunks in a stride-pattern fashion. For each message, $t \in \{0, 1, \cdots, Z-1\}$, its corresponding row chunk index is calculated by
$$page\_addr(t, i j)=I^{new}_{col_{t}} \pmod {N_{rc}}$$

where
$$I^{new}_{col_{t}} = (I^{new}_{col_{0}}+t) \pmod{Z}$$
$$I^{new}_{col_{0}} = Z - S(i, j)$$

Furthermore, each aforementioned message $t$ is located in a certain orow chunk by a word address, that is
$$word\_addr(t, i, j) =\lfloor I^{new}_{col_t} / N_{rc} \rfloor$$

In [2]:
def msgPass_buffer_addr_gen(msgPass_sched: str, t: int, Z: int, shift_factor: int, N_rc: int):
	if(msgPass_sched=="stride_sched"):
		I_new_col_0 = Z-shift_factor
		I_new_col_t = (I_new_col_0+t) % Z
		page_addr=I_new_col_t % N_rc
		word_addr=math.floor(I_new_col_t / N_rc)
	return page_addr, word_addr


Configuration of the decoder architecture
- Parallelism in rows and columns of the target parity-check matrix (expanded from a given base matrix)
- To instantiate a container to emulate the message-pass buffer

In [14]:
Z=15
W_s=1
P_r=3
P_c=1
S_i_j = 0
S_i_plus1_j = 7
U = math.ceil(Z / (P_r*W_s)) # number of stride units
N_rc = math.ceil(Z / P_r) # number of row chunks in a submatrix, each of which contains N_{fg} stride fingers
msgPass_buffer_norm = np.zeros(shape=(N_rc*2+1, P_r), dtype=np.int32) # Depth: Region 0) N_rc num. of compV2C row chunks
																      #        Region 1) a blank row chunk as separator
																      #        Region 2) N_rc num. of permV2C row chunks

In [15]:
# 1) Page address (index of stride set)
# 2) Word address (finger index in a stride set)
page_addr = [0]*Z
word_addr = [0]*Z

def msgPass_buffer_permMsg_write(
		msgPass_sched: str,
		compMsg_vec: List[int], # Set of computed messages before getting (cyclic) shifted
		Z: int,
		shift_factor: int,
		N_rc: int,
		msgPass_buffer_inst: List[ List[int] ],
		permMsg_pageAddr_base: int # Base address of permuted messages region in msgPass_buffer
) -> List[ List[int] ]:
	for t in compMsg_vec:
		page_addr[t], word_addr[t] = msgPass_buffer_addr_gen(msgPass_sched="stride_sched", t=t, Z=Z, shift_factor=shift_factor, N_rc=N_rc)
		msgPass_buffer_inst[ page_addr[t]+permMsg_pageAddr_base ][ word_addr[t] ] = t
	return msgPass_buffer_inst


Let test an example that $S(i,j)=7$ and $S(i+1,j)=7$.

In [ ]:
S_i_j = 0
S_i_plus1_j = 7
initial_compMsg_vec = [t for t in range(Z)]

permV2C_base_addr_vec = [0]*2
permV2C_base_addr_vec[0] = 0 # tentative value for ease of simulation
permV2C_base_addr_vec[1] = N_rc+1 # tentative value for ease of simulation

# For S(i, j)
msgPass_buffer_norm = msgPass_buffer_permMsg_write(
		msgPass_sched="stride_sched",
		compMsg_vec=initial_compMsg_vec,
		Z=Z,
		shift_factor=S_i_j,
		N_rc=N_rc,
		msgPass_buffer_inst=msgPass_buffer_norm,
		permMsg_pageAddr_base=permV2C_base_addr_vec[0]
)

# For S(i+1, j)
msgPass_buffer_norm = msgPass_buffer_permMsg_write(
		msgPass_sched="stride_sched",
		compMsg_vec=initial_compMsg_vec,
		Z=Z,
		shift_factor=S_i_plus1_j,
		N_rc=N_rc,
		msgPass_buffer_inst=msgPass_buffer_norm,
		permMsg_pageAddr_base=permV2C_base_addr_vec[1]
)

print(msgPass_buffer_norm)

[[ 0  5 10]
 [ 1  6 11]
 [ 2  7 12]
 [ 3  8 13]
 [ 4  9 14]
 [ 0  0  0]
 [ 7 12  2]
 [ 8 13  3]
 [ 9 14  4]
 [10  0  5]
 [11  1  6]]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


# Generation of Row-Chunk Cyclic Shift Factors

This section is to determine the cyclic shift factor for the permutation of each row chunk.
Let normalise the $S(i, j)$ and $S(i+1, j)$ with respect to the row chunks of which partitioned from a zero-shifted submatrix ($=I^{0}_{Z}$), denoted by $\hat{S}_{i, j, r}$ and $\hat{S}_{i, j, r}$, respectively. The last argument $r$ accounts for $r$-th row chunk in the underlying submatrix.
$$
\hat{S}_{i, j, r} =  word\_addr(t, i, j) S(i, j) \pmod{N_{rc}} \\
\hat{S}_{i+1, j, r} = S(i+1, j) \pmod{N_{rc}}
$$
The 

# Stride Unit Assignment

The row chunks, stride units and input sources of a page alignment unit are formulated as follows. A submatrix $b$ consists of a set of stride units, $\mathbb{S}=\{\mathbb{S}_{0}, \mathbb{S}_{1}, \cdots. \mathbb{S}_{U-1}\}$ where $U = Z / (P_{r} \cdot W^{s})$ accounts for the number of stride units. For all $u \in \{0, \cdots, U-1\}$, a set of consecutive row chunks is included in a stride unit, i.e. $\mathbb{S}_{u} = \{R^{u}_{0}, R^{u}_{2}, \cdots, R^{u}_{W^{s}-1}\}$ where $R^{u}_{0}$ denotes the first row chunk in the $u$th stride unit, etc. Moreover, a row chunk $R^{u}_{i}$ aggregates a bunch of extrinsic messages\footnote{Every extrinsic message is from a nonzero element in submatrix $b$, which represents one associated variable node.}, i.e.